# Julia 深度學習：卷積神經網路模型簡介

## 作業 033：訓練 CNN 學習門牌號碼資料集

訓練一個 CNN 模型來學習門牌號碼資料集。

In [1]:
using Flux
using Flux.Data: DataLoader
using Flux: @epochs, onecold, onehotbatch, throttle, logitcrossentropy
using MLDatasets
using Statistics

## 讀取資料

In [2]:
train_X, train_y = SVHN2.traindata(Float32, 1:20000)
test_X,  test_y  = SVHN2.testdata(Float32, 1:2000)

(Float32[0.14901961 0.15294118 … 0.19607843 0.1882353; 0.15294118 0.15294118 … 0.2 0.1882353; … ; 0.16470589 0.16862746 … 0.1764706 0.17254902; 0.15294118 0.15294118 … 0.16470589 0.16470589]

Float32[0.40392157 0.40784314 … 0.45882353 0.4509804; 0.40784314 0.40784314 … 0.4627451 0.4509804; … ; 0.40392157 0.39607844 … 0.45490196 0.4509804; 0.38039216 0.38039216 … 0.44313726 0.44313726]

Float32[0.23529412 0.23921569 … 0.29803923 0.2901961; 0.23921569 0.23921569 … 0.3019608 0.2901961; … ; 0.24313726 0.24705882 … 0.28235295 0.2784314; 0.22352941 0.22352941 … 0.27058825 0.2784314]

Float32[0.5058824 0.5254902 … 0.5411765 0.5137255; 0.49803922 0.52156866 … 0.50980395 0.47843137; … ; 0.48235294 0.49411765 … 0.39607844 0.43529412; 0.48235294 0.49019608 … 0.4392157 0.48235294]

Float32[0.5568628 0.5882353 … 0.59607846 0.5686275; 0.56078434 0.58431375 … 0.5647059 0.53333336; … ; 0.5254902 0.5372549 … 0.41960785 0.4627451; 0.5294118 0.5372549 … 0.4627451 0.50980395]

Float32[0.6 0.627451 … 0.647

In [3]:
train_y = onehotbatch(train_y, 1:10)
test_y = onehotbatch(test_y, 1:10)

10×2000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 0  0  1  0  0  1  0  1  1  0  0  0  0  …  0  0  0  0  0  0  1  0  0  0  1  0
 0  1  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  1  0  0  0  0  0  1  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  1     0  0  0  0  1  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  0  1  0  …  0  0  0  1  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0     0  1  0  0  0  0  0  1  0  1  0  0
 0  0  0  0  0  0  1  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0

In [4]:
batchsize = 1024
train = DataLoader(train_X, train_y, batchsize=batchsize, shuffle=true)
test = DataLoader(test_X, test_y, batchsize=batchsize)

DataLoader((Float32[0.14901961 0.15294118 … 0.19607843 0.1882353; 0.15294118 0.15294118 … 0.2 0.1882353; … ; 0.16470589 0.16862746 … 0.1764706 0.17254902; 0.15294118 0.15294118 … 0.16470589 0.16470589]

Float32[0.40392157 0.40784314 … 0.45882353 0.4509804; 0.40784314 0.40784314 … 0.4627451 0.4509804; … ; 0.40392157 0.39607844 … 0.45490196 0.4509804; 0.38039216 0.38039216 … 0.44313726 0.44313726]

Float32[0.23529412 0.23921569 … 0.29803923 0.2901961; 0.23921569 0.23921569 … 0.3019608 0.2901961; … ; 0.24313726 0.24705882 … 0.28235295 0.2784314; 0.22352941 0.22352941 … 0.27058825 0.2784314]

Float32[0.5058824 0.5254902 … 0.5411765 0.5137255; 0.49803922 0.52156866 … 0.50980395 0.47843137; … ; 0.48235294 0.49411765 … 0.39607844 0.43529412; 0.48235294 0.49019608 … 0.4392157 0.48235294]

Float32[0.5568628 0.5882353 … 0.59607846 0.5686275; 0.56078434 0.58431375 … 0.5647059 0.53333336; … ; 0.5254902 0.5372549 … 0.41960785 0.4627451; 0.5294118 0.5372549 … 0.4627451 0.50980395]

Float32[0.6 0.627

## CNN 模型

In [5]:
model = Chain(
    Conv((3, 3), 3=>32, relu; pad=1),
    MaxPool((2,2)),
    Conv((3, 3), 32=>32, relu; pad=1),
    MaxPool((2,2)),
    Conv((3, 3), 32=>64, relu; pad=1),
    MaxPool((2,2)),
    flatten,
    Dense(1024, 256, relu),
    Dense(256, 10),
    softmax)

Chain(Conv((3, 3), 3=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 32=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 32=>64, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), flatten, Dense(1024, 256, relu), Dense(256, 10), softmax)

## 使用 CUDA

In [ ]:
using CuArrays
model = model |> gpu
train_X = train_X |> gpu
train_y = train_y |> gpu
test_X = test_X |> gpu
test_y = test_y |> gpu

## 損失函數

In [6]:
loss(x, y) = logitcrossentropy(model(x), y)

loss (generic function with 1 method)

## Callback 函式

In [7]:
function test_loss()
    l = 0f0
    for (x, y) in test
        l += loss(x, y)
    end
    l/length(test)
end

test_loss (generic function with 1 method)

In [8]:
evalcb() = @show(test_loss())

evalcb (generic function with 1 method)

## 模型訓練

In [9]:
epochs = 50
@epochs epochs Flux.train!(loss, params(model), train, ADAM(0.005), cb=throttle(evalcb, 10))

┌ Info: Epoch 1
└ @ Main C:\Users\a504082002\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.2646604f0
test_loss() = 2.271379f0
test_loss() = 2.272058f0
test_loss() = 2.2720628f0
test_loss() = 2.2720506f0
test_loss() = 2.2719982f0
test_loss() = 2.2716827f0


┌ Info: Epoch 2
└ @ Main C:\Users\a504082002\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.273253f0
test_loss() = 2.2633407f0
test_loss() = 2.2547014f0
test_loss() = 2.2505536f0
test_loss() = 2.2484372f0
test_loss() = 2.2545185f0
test_loss() = 2.246439f0


┌ Info: Epoch 3
└ @ Main C:\Users\a504082002\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


...

test_loss() = 1.9052148f0
test_loss() = 1.8949215f0
test_loss() = 1.8959666f0
test_loss() = 1.8952007f0
test_loss() = 1.8952806f0
test_loss() = 1.896979f0
test_loss() = 1.8969905f0
test_loss() = 1.8992834f0


┌ Info: Epoch 49
└ @ Main C:\Users\a504082002\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 1.8938956f0
test_loss() = 1.8920286f0
test_loss() = 1.8973808f0
test_loss() = 1.9015036f0
test_loss() = 1.902606f0
test_loss() = 1.8954872f0
test_loss() = 1.8943868f0
test_loss() = 1.8955756f0
test_loss() = 1.8976896f0
test_loss() = 1.8965492f0


┌ Info: Epoch 50
└ @ Main C:\Users\a504082002\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 1.8979628f0
test_loss() = 1.9015949f0
test_loss() = 1.8988416f0
test_loss() = 1.8968601f0
test_loss() = 1.8978627f0
test_loss() = 1.897214f0
test_loss() = 1.8916422f0
test_loss() = 1.8940676f0
test_loss() = 1.8948131f0


## 模型評估

In [10]:
accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))

accuracy (generic function with 1 method)

In [11]:
accuracy(test_X, test_y)

0.564